In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [3]:
files = glob('forums/*.json')

In [4]:
forums = []
for f in files:
    with open(f) as fopen:
        forums.extend(json.load(fopen))
        
forums = sorted(list(set(forums)))
len(forums)

96333

In [10]:
forums[3]

'https://www.mycarforum.com/forums/topic/1000510-which-is-better'

In [21]:
def get_href(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    try:
        max_page = [a_.text for a_ in a if 'Page 1 of ' in a_.text][0]
        max_page = int(max_page.split('Page 1 of ')[-1].split()[-1])
    except:
        max_page = -1
    
    div = soup.find_all('div', {'data-role': "commentContent"})
        
    return url, [str(d) for d in div], max_page

In [22]:
get_href(forums[3])

('https://www.mycarforum.com/forums/topic/1000510-which-is-better',
 ['<div class="ipsType_normal ipsType_richText ipsPadding_bottom ipsContained" data-controller="core.front.core.lightboxedImages" data-role="commentContent">\n<p>Ford Focus or Renault Megane?</p>\n</div>',
  '<div class="ipsType_normal ipsType_richText ipsPadding_bottom ipsContained" data-controller="core.front.core.lightboxedImages" data-role="commentContent">\n<p>Seems like the Focus got better reviews cos it has ESP and some other gadgets..</p>\n</div>',
  '<div class="ipsType_normal ipsType_richText ipsPadding_bottom ipsContained" data-controller="core.front.core.lightboxedImages" data-role="commentContent">\n<p>Hmm, tough choice there.  The megane feels more powerful and pickup is faster than the Focus. Both cars handles very well with nice balance even over uneven road surfaces (tried the Focus along Kim Seng Road and AYE while the Megane along the winding road with metal plates coming down from Raffles Hotel to 

In [23]:
forums[3]

'https://www.mycarforum.com/forums/topic/1000510-which-is-better'

In [24]:
!rm -rf threads
!mkdir threads

In [ ]:
max_worker = 2

for i in tqdm(range(0, len(forums), max_worker)):
    
    filename = os.path.join('threads', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
        
    topics = [url for url in forums[i: i + max_worker]]
    
    data = []    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href, url): url for url in topics}

        for future in as_completed(futures):
            try:
                data.append(future.result())
            except:
                pass
                
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)
        
    try:
        del data
        del futures
    except:
        pass

  9%|███                              | 4444/48167 [1:21:15<19:16:40,  1.59s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 25%|███████▉                        | 11912/48167 [3:41:20<11:20:19,  1.13s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 41%|█████████████▌                   | 19765/48167 [6:06:22<6:54:21,  1.14it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t